In [1]:
!pip install pandas
!pip install plotly

!pip install scikit-learn # non-depreaceated sklearn


[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


Task 1

In [2]:
import pandas as pd
# read the csv and make it a pandas.DataFrame
frame = pd.DataFrame(pd.read_csv("./online_shoppers_intention.csv"))
print(frame)

       Administrative  Administrative_Duration  Informational  \
0                   0                      0.0              0   
1                   0                      0.0              0   
2                   0                      0.0              0   
3                   0                      0.0              0   
4                   0                      0.0              0   
...               ...                      ...            ...   
12325               3                    145.0              0   
12326               0                      0.0              0   
12327               0                      0.0              0   
12328               4                     75.0              0   
12329               0                      0.0              0   

       Informational_Duration  ProductRelated  ProductRelated_Duration  \
0                         0.0               1                 0.000000   
1                         0.0               2                64.000000 

In [ ]:
#visualisation

Verhaaltje van ~1min

Task 2

In [ ]:
#Prepositioning op basis van algoritmes in task 3

Task 3

In [ ]:
# affinity propagation clustering

# DBSCAN clustering

# Birch clustering

Task 4

In [ ]:
# manual Silhouette score

# Davies Buildin Score

# calinski-harabasz index

Task 5 ALL MANUAL


In [ ]:
# Euclidian distance

# Manhattan distance

# Cosine similarity

# Distances effect on dbscan